In [3]:
import os
import glob
import numpy as np
import pandas as pd
import PIL.Image
import h5py
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.applications
import tensorflow.keras.layers as layers
import keras.backend as k
from tensorflow.keras.applications.mobilenet import preprocess_input
from io import BytesIO
import base64

""" make sure this notebook is running from root directory """
while os.path.basename(os.getcwd()) in ('notebooks', 'src'):
    os.chdir('..')
assert ('README.md' in os.listdir('./')), 'Can not find project root, please cd to project root before running the following code'

In [4]:
path_model_save = './asset_model/cnn_face_attr_celeba'

In [ ]:
###################################
# THIS IS ACTUALLY NOT NECESSARY
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.28

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))
###################################

In [6]:
def create_cnn_model(size_output=None, tf_print=False):
    """
    create keras model with convolution layers of MobileNet and added fully connected layers on to top
    :param size_output: number of nodes in the output layer
    :param tf_print:    True/False to print
    :return: keras model object
    """

    if size_output is None:
        # MAKE SURE TO PASS IN HARDCODED SIZE - DON'T HAVE CSV DATA FILES
        # get number of attrubutes, needed for defining the final layer size of network
        df_attr = pd.read_csv(path_celeba_att, sep='\s+', header=1, index_col=0)
        size_output = df_attr.shape[1]

    # Load the convolutional layers of pretrained model: mobilenet
    base_model = tensorflow.keras.applications.mobilenet.MobileNet(
        include_top=False, input_shape=(128,128,3),
        alpha=1, depth_multiplier=1,
        dropout=0.001, weights="imagenet",
        input_tensor=None, pooling=None)

    # add fully connected layers
    fc0 = base_model.output
    fc0_pool = layers.GlobalAveragePooling2D(data_format='channels_last', name='fc0_pool')(fc0)
    fc1 = layers.Dense(256, activation='relu', name='fc1_dense')(fc0_pool)
    fc2 = layers.Dense(size_output, activation='tanh', name='fc2_dense')(fc1)

    model = tensorflow.keras.models.Model(inputs=base_model.input, outputs=fc2)

    # freeze the early layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='sgd', loss='mean_squared_error')

    if tf_print:
        print('use convolution layers of MobileNet, add fully connected layers')
        print(model.summary())

    return model

def get_list_model_save(path_model_save=path_model_save):
    return glob.glob(os.path.join(path_model_save, 'model*.h5'))

In [7]:
model = create_cnn_model(size_output=40)
model.load_weights(get_list_model_save()[-1])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
__________

In [21]:
def image_to_feature(image_path, model=model):
    img = np.asarray(PIL.Image.open(image_path).resize((128, 128), resample=PIL.Image.BILINEAR))
    x = np.stack([img], axis=0)
    x = x[:,:,:,:3]
    print(x.shape)
    print(x[:, :2, :2, :])
    return model.predict(preprocess_input(x), batch_size=32)

def b64_image_to_feature(b64_str, model=model):
    image_data = re.sub('^data:image/.+;base64,', '', b64_str)
    img = np.asarray(PIL.Image.open(BytesIO(base64.b64decode(image_data))).resize((128, 128), resample=PIL.Image.BILINEAR))
    x = np.stack([img], axis=0)
    return model.predict(preprocess_input(x), batch_size=32)

In [22]:
np.array(image_to_feature("data/processed/cyberextender/000420/000007.jpg")[0])
#np.array(image_to_feature("src/notebooks/Profile_YisongYue_156x150.jpg")[0])

(1, 128, 128, 3)
[[[[192 200 203]
   [192 200 203]]

  [[192 200 202]
   [192 200 202]]]]


array([-0.95837665, -0.9420335 ,  0.11940438, -0.30442753, -0.99999946,
       -0.9999996 , -0.7525677 ,  0.5638125 ,  0.7467824 , -0.99928004,
       -0.9999967 , -0.9972268 ,  0.74412036, -0.9978278 , -0.9988206 ,
       -0.9999995 , -0.99997133, -0.99987286, -0.98980856, -0.10975574,
        0.996792  ,  0.79417807, -0.999932  , -0.9641402 ,  0.9906429 ,
       -0.6010365 , -0.99996775, -0.84928405, -0.9913011 , -0.9998905 ,
       -0.9999994 ,  0.9119817 ,  0.12061686, -0.9988287 , -0.9971764 ,
       -0.9999827 , -0.9774961 , -0.8852496 , -0.9921849 ,  0.88984966],
      dtype=float32)

In [7]:
feature_names = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
feature_list= []
for i, img_name in enumerate(glob.glob("./data/processed/UTKFace/*.jpg")):
    feature_list.append(image_to_feature(img_name)[0])
    if i % 100 == 0:
        print(i)

df = pd.DataFrame(feature_list, columns=feature_names)
df.to_csv("./data/processed/UTKFace_features.csv")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


KeyboardInterrupt: 

In [20]:
print(df.head())

   5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes      Bald  \
0         -1.000000        -0.728840    0.977653        -0.967019 -1.000000   
1         -0.999982        -0.997856   -0.228717        -0.953192 -1.000000   
2         -0.996685        -0.973225   -0.416376        -0.806436 -0.999998   
3         -1.000000        -0.915146    0.914741        -0.994652 -1.000000   
4         -0.979455         0.824346    0.900790        -0.273816 -0.999992   

      Bangs  Big_Lips  Big_Nose  Black_Hair  Blond_Hair    ...     Sideburns  \
0 -0.999956 -0.707431 -0.956450   -0.984351   -0.958941    ...     -1.000000   
1 -0.947790 -0.991040 -0.995475    0.638358   -0.999686    ...     -1.000000   
2 -0.999949  0.536399  0.712077   -0.999485   -0.999620    ...     -0.999995   
3 -1.000000 -0.975020 -0.979644   -0.995063   -0.938367    ...     -1.000000   
4 -0.952782  0.313058 -0.193981   -0.956924   -0.962793    ...     -1.000000   

    Smiling  Straight_Hair  Wavy_Hair  Weari